# Estimate Binomial draw probabilility

In [1]:
using DynamicHMCModels

Define a structure to hold the data.

In [2]:
struct BernoulliProblem
    "Total number of draws in the data."
    n::Int
    "Number of draws `==1` in the data"
    s::Vector{Int}
end;

Make the type callable with the parameters *as a single argument*.

In [3]:
function (problem::BernoulliProblem)((α, )::NamedTuple{(:α, )})
    @unpack n, s = problem        # extract the data
    loglikelihood(Binomial(n, α), s)
end

Create the data and complete setting up the problem.

In [4]:
obs = rand(Binomial(9, 2/3), 1)
p = BernoulliProblem(9, obs)
p((α = 0.5, ))

-2.6548056865833978

Write a function to return properly dimensioned transformation.

In [5]:
problem_transformation(p::BernoulliProblem) =
    as((α = as𝕀, ),  )
# Use a flat priors (the default, omitted) for α
P = TransformedLogDensity(problem_transformation(p), p)
∇P = LogDensityRejectErrors(ADgradient(:ForwardDiff, P));

Sample

In [6]:
chain, NUTS_tuned = NUTS_init_tune_mcmc(∇P, 1000)

MCMC, adapting ϵ (75 steps)
1.4e-5 s/step ...done
MCMC, adapting ϵ (25 steps)
1.2e-5 s/step ...done
MCMC, adapting ϵ (50 steps)
1.6e-5 s/step ...done
MCMC, adapting ϵ (100 steps)
1.8e-5 s/step ...done
MCMC, adapting ϵ (200 steps)
1.7e-5 s/step ...done
MCMC, adapting ϵ (400 steps)
1.7e-5 s/step ...done
MCMC, adapting ϵ (50 steps)
1.4e-5 s/step ...done
MCMC (1000 steps)
3.3e-5 s/step ...done


(NUTS_Transition{Array{Float64,1},Float64}[NUTS_Transition{Array{Float64,1},Float64}([-0.19382717397516994], -4.716916021027982, 1, DynamicHMC.DoubledTurn, 0.5, 2), NUTS_Transition{Array{Float64,1},Float64}([-0.2450237162030019], -4.976412868723488, 2, DynamicHMC.DoubledTurn, 0.7443464434820126, 4), NUTS_Transition{Array{Float64,1},Float64}([0.293905614608316], -6.342118275480957, 1, DynamicHMC.AdjacentTurn, 0.7209087240941228, 4), NUTS_Transition{Array{Float64,1},Float64}([0.7912462007265633], -3.6336457118874037, 2, DynamicHMC.AdjacentTurn, 0.830594009237764, 6), NUTS_Transition{Array{Float64,1},Float64}([2.202976890841579], -4.447496930489268, 3, DynamicHMC.AdjacentTurn, 0.8699755258412372, 10), NUTS_Transition{Array{Float64,1},Float64}([-0.017709938234090783], -4.982933797593993, 3, DynamicHMC.AdjacentTurn, 0.8934493657092467, 10), NUTS_Transition{Array{Float64,1},Float64}([0.2922361782701877], -4.519423968016369, 1, DynamicHMC.AdjacentTurn, 0.7417036150618147, 4), NUTS_Transition{

To get the posterior for ``α`` use `get_position` and then transform back.

In [7]:
posterior = TransformVariables.transform.(Ref(problem_transformation(p)), get_position.(chain));

Extract the parameter.

In [8]:
posterior_α = first.(posterior);

check the effective sample size

In [9]:
ess_α = effective_sample_size(posterior_α)

287.0542772652134

NUTS-specific statistics

In [10]:
NUTS_statistics(chain)

Hamiltonian Monte Carlo sample of length 1000
  acceptance rate mean: 0.8, min/25%/median/75%/max: 0.49 0.75 0.86 0.87 0.94
  termination: AdjacentTurn => 56% DoubledTurn => 44%
  depth: 1 => 24% 2 => 43% 3 => 22% 4 => 11%


check the mean

In [11]:
mean(posterior_α)

0.7208820297411053

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*